# NUEVOS DATOS DE NIGERIA

link al dataset: https://zenodo.org/record/1252141

In [49]:
import pandas as pd
import glob
import ssqueezepy #para convertir la señal en imagen
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import mne #eeg analysis library
import scipy.io
#import torch.nn as nn
#import torch
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [50]:
metadata = pd.read_csv('C:\\Users\\47575909\\Desktop\\metadata_nigeria.csv')
metadata

,subject.id,recordedPeriod,startTime,session.id,first_condition,remarks,Group,csv.file
0,6,270,26/9/2016 13:13,1,open,NaN,control,signal-6-1.csv.gz
1,9,271,26/9/2016 13:30,1,closed,NaN,control,signal-9-1.csv.gz
2,10,272,26/9/2016 13:36,1,open,eyes closed at 2:40,control,signal-10-1.csv.gz
3,11,274,26/9/2016 13:42,2,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-2.csv.gz
4,11,1,26/9/2016 13:42,1,closed,no.11.1 failed >> 11.2 is the right one,control,signal-11-1.csv.gz
5,12,271,26/9/2016 13:47,1,open,NaN,control,signal-12-1.csv.gz
6,40,271,26/9/2016 16:56,1,open,eyes closed at 2:40,control,signal-40-1.csv.gz
7,49,268,27/9/2016 14:29,1,closed,Teacher + not deaf but sign language experienc...,control,signal-49-1.csv.gz
8,63,268,30/9/2016 9:45,1,open,Oxipitaal slecht contact eerste 2:19 minuten,control,signal-63-1.csv.gz
9,64,271,30/9/2016 9:51,1,closed,NaN,control,signal-64-1.csv.gz


In [51]:
#veo la mierda de los ids
groups = metadata.groupby('subject.id')

grp_keys = list(groups.groups.keys())
print(len(grp_keys))

222


In [52]:
metadata = metadata.drop([4, 10, 11], axis=0)  # Elimina las filas en las posiciones 4, 10 y 11
metadata.at[64, 'subject.id'] = "124-2"  # Actualiza el valor en la fila 64 de la columna 'subject.id'
metadata.reset_index(drop=True, inplace=True)  # Restablece los índices y aplica los cambios

metadata
#SINO ELIMINA ESA PERSONA Y YA ESTA QCY

ValueError: invalid literal for int() with base 10: '124-2'

In [32]:
metadata = metadata.drop('startTime', axis=1)
metadata = metadata.drop('first_condition', axis=1)
metadata = metadata.drop('recordedPeriod', axis=1)
metadata = metadata.drop('remarks', axis=1)
metadata = metadata.drop('session.id', axis=1)

metadata['csv.file'] = metadata['csv.file'].str.replace('.csv.gz', '.csv')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(metadata)

     subject.id     Group          csv.file
0             6   control    signal-6-1.csv
1             9   control    signal-9-1.csv
2            10   control   signal-10-1.csv
3            11   control   signal-11-2.csv
4            12   control   signal-12-1.csv
5            40   control   signal-40-1.csv
6            49   control   signal-49-1.csv
7            63   control   signal-63-1.csv
8            64   control   signal-64-1.csv
9            65   control   signal-65-2.csv
10           65   control   signal-65-1.csv
11           66   control   signal-66-1.csv
12           67   control   signal-67-1.csv
13           68   control   signal-68-1.csv
14           69   control   signal-69-1.csv
15           70   control   signal-70-1.csv
16           71   control   signal-71-1.csv
17           72   control   signal-72-1.csv
18           73   control   signal-73-1.csv
19           74   control   signal-74-1.csv
20           75   control   signal-75-1.csv
21           76   control   sign

In [6]:
'''dataset = pd.DataFrame()

for archivo_csv in os.listdir('C:\\Users\\47575909\\Desktop\\EEGs_Nigeria'):
    if archivo_csv.endswith('.csv'):
        archivo_csv = archivo_csv.replace(' ', '')
        
        # Obtener el nombre de archivo sin la extensión .csv
        archivo_base = archivo_csv.replace('.csv', '')

        # Verifica si hay filas en metadata que cumplen con la condición de búsqueda
        matching_rows = metadata.loc[metadata['csv.file'] == archivo_base]

        if not matching_rows.empty:  # Verifica si se encontraron coincidencias
            subject_id, label = matching_rows[['subject.id', 'Group']].values[0]

            # Leer el archivo CSV de EEG
            eeg_data = pd.read_csv(os.path.join(directorio_eeg, archivo_csv))

            # Agregar las columnas de subject ID y etiqueta al DataFrame de EEG
            eeg_data['subject.id'] = subject_id
            eeg_data['Group'] = label

            # Agregar los datos del EEG al conjunto de datos combinado
            dataset = pd.concat([dataset, eeg_data])

            print(f"Se encontró una coincidencia para {archivo_csv}: subject_id={subject_id}, Group={label}")
        else:
            print(f"No se encontraron coincidencias para {archivo_csv}")

# Puedes guardar el conjunto de datos combinado en un nuevo archivo CSV si lo deseas
dataset.to_csv('new_eeg_data.csv', index=False)'''

'dataset = pd.DataFrame()\n\nfor archivo_csv in os.listdir(\'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\'):\n    if archivo_csv.endswith(\'.csv\'):\n        archivo_csv = archivo_csv.replace(\' \', \'\')\n        \n        # Obtener el nombre de archivo sin la extensión .csv\n        archivo_base = archivo_csv.replace(\'.csv\', \'\')\n\n        # Verifica si hay filas en metadata que cumplen con la condición de búsqueda\n        matching_rows = metadata.loc[metadata[\'csv.file\'] == archivo_base]\n\n        if not matching_rows.empty:  # Verifica si se encontraron coincidencias\n            subject_id, label = matching_rows[[\'subject.id\', \'Group\']].values[0]\n\n            # Leer el archivo CSV de EEG\n            eeg_data = pd.read_csv(os.path.join(directorio_eeg, archivo_csv))\n\n            # Agregar las columnas de subject ID y etiqueta al DataFrame de EEG\n            eeg_data[\'subject.id\'] = subject_id\n            eeg_data[\'Group\'] = label\n\n            # Agregar los da

In [7]:
noseque = pd.read_csv('C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-10-1.csv')
noseque

,Unnamed: 0,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,...,CQ_F3,CQ_P7,CQ_P8,CQ_F4,CQ_AF3,CQ_FC5,CQ_O1,CQ_T8,CQ_F8,CQ_DRL
0,1,4004,-227.315789,4045,106.126316,124.989474,3705,321.400000,3866,4093,...,4,4,4,4,4,4,4,4,4,4
1,2,3998,-230.400000,4043,94.842105,121.400000,3698,316.273684,3862,4092,...,4,4,4,4,4,4,4,4,4,4
2,3,3997,-239.631579,4045,92.273684,120.884211,3696,314.221053,3861,4081,...,4,4,4,4,4,4,4,4,4,4
3,4,4005,-243.221053,4046,101.505263,125.494737,3701,320.368421,3863,4142,...,4,4,4,4,4,4,4,4,4,4
4,5,4003,-241.168421,4044,100.484211,127.547368,3699,323.957895,3860,4244,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34811,34812,3902,82.536842,4106,199.452632,90.115789,3622,-338.715789,3896,4326,...,4,4,4,4,4,4,4,4,4,4
34812,34813,3894,75.863158,4104,200.989474,88.578947,3618,-338.715789,3896,4326,...,4,4,4,4,4,4,4,4,4,4
34813,34814,3903,77.400000,4099,202.536842,91.652632,3632,-339.221053,3909,4323,...,4,4,4,4,4,4,4,4,4,4
34814,34815,3913,86.631579,4104,206.631579,92.168421,3644,-341.789474,3914,4321,...,4,4,4,4,4,4,4,4,4,4


In [8]:
path = 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria'
all_files = glob.glob(os.path.join(path, "*.csv"))

def alfanumerico_key(item):
    partes = re.split(r'(\d+)', item)
    partes[1::2] = map(int, partes[1::2])
    return partes

all_files = sorted(all_files, key=alfanumerico_key)

def add_id_y(all_files, metadata):
    modified_dataframes_list = []

    for filename, subject_id, group in zip(all_files, metadata['subject.id'], metadata['Group']):
        df = pd.read_csv(filename, index_col=None, header=0)
        dataframe = df.iloc[:, 1:15].copy()
        # Obtén el número del archivo a partir del nombre
        file_number = int(re.search(r'(\d+)', os.path.basename(filename)).group(1))
        dataframe['subject_id'] = subject_id
        dataframe['y'] = group
        modified_dataframes_list.append(dataframe)

    return modified_dataframes_list

In [9]:
all_files

['C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-6-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-9-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-10-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-11-2.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-12-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-40-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-49-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-63-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-64-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-66-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-67-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-68-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-70-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-71-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-72-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Niger

In [10]:
modified_dataframes = add_id_y(all_files, metadata)

dataframes_list = []

for dataframe in modified_dataframes:
    dataframes_list.append(dataframe)

all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
all_dataframes.to_csv('eeg_new_data.csv', index=False)

In [11]:
all_dataframes

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4170,-197.063158,4082,188.168421,126.526316,3798,-237.168421,3943,3957,-13.473684,-132.957895,4168,3818,-110.400000,6,control
1,4171,-192.957895,4082,188.684211,126.010526,3803,-240.768421,3951,3957,-1.684211,-135.526316,4172,3817,-103.221053,6,control
2,4171,-193.989474,4078,190.736842,121.905263,3805,-244.357895,3951,3956,-4.757895,-135.526316,4173,3813,-102.189474,6,control
3,4169,-198.600000,4076,187.147368,120.368421,3795,-244.863158,3942,3956,-19.631579,-135.010526,4168,3814,-110.915789,6,control
4,4168,-200.652632,4075,183.042105,121.400000,3790,-244.863158,3937,3955,-28.863158,-135.526316,4166,3816,-117.063158,6,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7342587,4138,-283.221053,4202,-341.168421,171.652632,3817,-289.484211,4013,4249,158.315789,-85.778947,4221,3835,-99.631579,635,epilepsy
7342588,4141,-276.547368,4202,-337.578947,173.189474,3817,-291.021053,4014,4249,156.778947,-86.810526,4229,3833,-94.505263,635,epilepsy
7342589,4137,-280.136842,4202,-342.189474,176.778947,3818,-294.094737,4010,4249,159.347368,-85.273684,4227,3835,-99.115789,635,epilepsy
7342590,4144,-268.863158,4205,-336.547368,180.884211,3832,-293.073684,4020,4249,169.084211,-78.094737,4232,3845,-93.473684,635,epilepsy


In [12]:
prueba = pd.read_csv('C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-638-1.csv')
prueba

,Unnamed: 0,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,...,CQ_F3,CQ_P7,CQ_P8,CQ_F4,CQ_AF3,CQ_FC5,CQ_O1,CQ_T8,CQ_F8,CQ_DRL
0,1,4098,326.631579,4231,250.736842,158.315789,3725,-261.789474,3970,4350,...,4,4,4,4,4,4,4,4,4,4
1,2,4097,326.126316,4230,246.631579,159.863158,3719,-262.305263,3958,4344,...,4,4,4,4,4,4,4,4,4,4
2,3,4097,333.305263,4229,252.273684,161.905263,3726,-260.252632,3963,4345,...,4,4,4,4,4,4,4,4,4,4
3,4,4101,339.452632,4230,259.452632,163.452632,3738,-258.200000,3975,4350,...,4,4,4,4,4,4,4,4,4,4
4,5,4108,344.578947,4232,263.557895,166.010526,3742,-257.168421,3975,4350,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34939,34940,4138,-283.221053,4202,-341.168421,171.652632,3817,-289.484211,4013,4249,...,4,4,4,4,4,4,4,4,4,4
34940,34941,4141,-276.547368,4202,-337.578947,173.189474,3817,-291.021053,4014,4249,...,4,4,4,4,4,4,4,4,4,4
34941,34942,4137,-280.136842,4202,-342.189474,176.778947,3818,-294.094737,4010,4249,...,4,4,4,4,4,4,4,4,4,4
34942,34943,4144,-268.863158,4205,-336.547368,180.884211,3832,-293.073684,4020,4249,...,4,4,4,4,4,4,4,4,4,4


In [20]:
id_list = sorted(all_dataframes['subject_id'].unique())

print(id_list)

[6, 9, 10, 11, 12, 40, 49, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 116, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625,